<a href="https://colab.research.google.com/github/Guiils/analise_de_dados/blob/main/tratamento_clima_mensal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparação de bibliotecas

In [ ]:
pip install pyspark

In [ ]:
pip install gcsfs

In [ ]:
pip install pandera

In [ ]:

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import isnull
from pyspark.sql.types import FloatType, DoubleType, StructType, StructField, StringType, IntegerType
from google.cloud import storage
import os
import pandera as pa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

##Conexões com GCP e Spark

In [ ]:
#CONFIGURANDO DA CHAVE DE SEGURANCA (Enviada com o projeto)

serviceAccount = '/content/loyal-optics-377819-ebe367c145ff.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#Configurações Google Cloud Storage
client = storage.Client()
bucket = client.get_bucket('tcc_soulcode_g3')

In [ ]:
#configurar a variavel de ambiente (sessão) Pyspark
spark = (SparkSession.builder
                     .master('local[3]')
                     .appName('tratramento_clima_mensal')
                     .config('spark.ui.port', '4050')
                     .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                     .getOrCreate()
)

In [ ]:
spark

##Extraindo data frame da Cloud

In [ ]:

schema = StructType([
    StructField("Data Medicao", StringType(), True),
    StructField("NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)", StringType(), True),
    StructField("PRECIPITACAO TOTAL - MENSAL (AUT)(mm)", StringType(), True),
    StructField("TEMPERATURA MEDIA - MENSAL (AUT)(°C)", StringType(), True),
    StructField("VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)", StringType(), True),
    StructField("VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)", StringType(), True),
    StructField("MUNICIPIO", StringType(), True),
    StructField("LATITUDE", StringType(), True),
    StructField("LONGITUDE", StringType(), True)
])

path = 'gs://tcc_soulcode_g3/brutos_clima_mensal/sul/dados_A899_M_2010-01-01_2022-12-31.csv'

In [ ]:
#Importando dataframe do GCP
df = (
         spark.read
         .format('csv')
         .option('delimiter', ';') 
         .option('header', 'false')
         .schema(schema)
         .option('encoding', 'utf-8')
         .option('inferschema', 'true')
         .load(path)
)


##Analise exploratória

In [ ]:
df.show(truncate = False)

+---------------------------------------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+---------+--------+---------+
|Data Medicao                                 |NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)|PRECIPITACAO TOTAL - MENSAL (AUT)(mm)|TEMPERATURA MEDIA - MENSAL (AUT)(°C)|VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)|VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)|MUNICIPIO|LATITUDE|LONGITUDE|
+---------------------------------------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+---------+--------+---------+
|Nome: Santa Vitoria do Palmar - Barra do Chui|null                                              

In [ ]:
df_back = df

In [ ]:
df = df_back

##Tratamento

In [ ]:
for i in range(4):
  if i == 1:
    pass
  linha = df.collect()[i]
  coluna_separador = linha["Data Medicao"].split(": ")
  if i == 0:
    municipio = coluna_separador[1]
  if i == 2:
    latitude = coluna_separador[1]
  if i == 3:
    longitude = coluna_separador[1]

print(municipio, latitude, longitude)



Santa Vitoria do Palmar - Barra do Chui -33.74222221 -53.37222221


In [ ]:
df = df.withColumn('MUNICIPIO', F.lit(municipio))
df = df.withColumn('LATITUDE', F.lit(latitude))
df = df.withColumn('LONGITUDE', F.lit(longitude))

In [ ]:
df.show()

+--------------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+------------+------------+
|        Data Medicao|NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)|PRECIPITACAO TOTAL - MENSAL (AUT)(mm)|TEMPERATURA MEDIA - MENSAL (AUT)(°C)|VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)|VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)|           MUNICIPIO|    LATITUDE|   LONGITUDE|
+--------------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+------------+------------+
|Nome: Santa Vitor...|                                                       null|                                 nul

In [ ]:
df = df.subtract(df.limit(10))

In [ ]:
df.show()

+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+------------+------------+
|Data Medicao|NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)|PRECIPITACAO TOTAL - MENSAL (AUT)(mm)|TEMPERATURA MEDIA - MENSAL (AUT)(°C)|VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)|VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)|           MUNICIPIO|    LATITUDE|   LONGITUDE|
+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+------------+------------+
|  2018-04-30|                                                          7|                                   83|                             2

In [ ]:
for coluna in df.columns:
    # Verificar se a coluna é do tipo string
  if df.schema[coluna].dataType == StringType():
    # Substituir todas as ocorrências de "," por "."
    df = df.withColumn(coluna, F.regexp_replace(F.col(coluna), ",", "."))
    df = df.withColumn(coluna, F.regexp_replace(F.col(coluna), r'^\.', '0.'))

In [ ]:
df.show()

+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+------------+------------+
|Data Medicao|NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)|PRECIPITACAO TOTAL - MENSAL (AUT)(mm)|TEMPERATURA MEDIA - MENSAL (AUT)(°C)|VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)|VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)|           MUNICIPIO|    LATITUDE|   LONGITUDE|
+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+------------+------------+
|  2018-04-30|                                                          7|                                   83|                             2

In [ ]:
#Checando valores nulos nas colunas

for coluna in df.columns:
  nulos = df.filter(isnull(coluna)).count()
  print(f"Existem {nulos} valores nulos na coluna {coluna}.")

Existem 0 valores nulos na coluna Data Medicao.
Existem 0 valores nulos na coluna NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero).
Existem 0 valores nulos na coluna PRECIPITACAO TOTAL - MENSAL (AUT)(mm).
Existem 0 valores nulos na coluna TEMPERATURA MEDIA - MENSAL (AUT)(°C).
Existem 0 valores nulos na coluna VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h).
Existem 0 valores nulos na coluna VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h).
Existem 0 valores nulos na coluna MUNICIPIO.
Existem 0 valores nulos na coluna LATITUDE.
Existem 0 valores nulos na coluna LONGITUDE.


In [ ]:
lista_colunas_double = ['NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)','PRECIPITACAO TOTAL - MENSAL (AUT)(mm)','TEMPERATURA MEDIA - MENSAL (AUT)(°C)','VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)','VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)']
lista_km = ['VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)','VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)']
lista_int = ['NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)']

for coluna in df.columns:
  if coluna in lista_colunas_double:
    df = df.withColumn(coluna, F.col(coluna).cast(DoubleType()))
  if coluna in lista_km:
    df = df.withColumn(coluna, F.col(coluna) * 3.6)
  if coluna in lista_int:
    df = df.withColumn(coluna, F.col(coluna).cast(IntegerType()))   
    

In [ ]:
df.printSchema()

root
 |-- Data Medicao: string (nullable = true)
 |-- NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero): integer (nullable = true)
 |-- PRECIPITACAO TOTAL - MENSAL (AUT)(mm): double (nullable = true)
 |-- TEMPERATURA MEDIA - MENSAL (AUT)(°C): double (nullable = true)
 |-- VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h): double (nullable = true)
 |-- VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h): double (nullable = true)
 |-- MUNICIPIO: string (nullable = false)
 |-- LATITUDE: string (nullable = false)
 |-- LONGITUDE: string (nullable = false)



In [ ]:
# Dropando as colunas de latitude e longitude já que criamos a de coordenada com os dados juntos
df = df.withColumn('COORDENADA',F.concat_ws(', ',F.col('LATITUDE'),F.col('LONGITUDE'))).drop('LATITUDE', 'LONGITUDE')

In [ ]:
df_tratado = df.withColumn('Data Medicao', F.to_date(F.col('Data Medicao'), 'yyyy-MM-dd'))

In [ ]:
df_tratado.printSchema()

root
 |-- Data Medicao: date (nullable = true)
 |-- NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero): integer (nullable = true)
 |-- PRECIPITACAO TOTAL - MENSAL (AUT)(mm): double (nullable = true)
 |-- TEMPERATURA MEDIA - MENSAL (AUT)(°C): double (nullable = true)
 |-- VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h): double (nullable = true)
 |-- VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h): double (nullable = true)
 |-- MUNICIPIO: string (nullable = false)
 |-- COORDENADA: string (nullable = false)



In [ ]:
df_tratado.show()

+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+--------------------+
|Data Medicao|NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)|PRECIPITACAO TOTAL - MENSAL (AUT)(mm)|TEMPERATURA MEDIA - MENSAL (AUT)(°C)|VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)|VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)|           MUNICIPIO|          COORDENADA|
+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+--------------------+
|  2018-04-30|                                                          7|                                 83.0|                             21.6425|        

Checando os Valores de cada coluna

In [ ]:
total = df_tratado.count() - df_tratado.dropDuplicates().count()
print(f'Total de Registros Duplicados = {total}')

Total de Registros Duplicados = 0


In [ ]:
#Checando valores nulos nas colunas 

for coluna in df_tratado.columns:
  nulos = df_tratado.filter(isnull(coluna)).count()
  print(f"Existem {nulos} valores nulos na coluna {coluna}.")

Existem 0 valores nulos na coluna Data Medicao.
Existem 21 valores nulos na coluna NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero).
Existem 29 valores nulos na coluna PRECIPITACAO TOTAL - MENSAL (AUT)(mm).
Existem 18 valores nulos na coluna TEMPERATURA MEDIA - MENSAL (AUT)(°C).
Existem 17 valores nulos na coluna VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h).
Existem 20 valores nulos na coluna VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h).
Existem 0 valores nulos na coluna MUNICIPIO.
Existem 0 valores nulos na coluna COORDENADA.


In [ ]:
df_tratado.printSchema()

root
 |-- Data Medicao: date (nullable = true)
 |-- NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero): integer (nullable = true)
 |-- PRECIPITACAO TOTAL - MENSAL (AUT)(mm): double (nullable = true)
 |-- TEMPERATURA MEDIA - MENSAL (AUT)(°C): double (nullable = true)
 |-- VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h): double (nullable = true)
 |-- VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h): double (nullable = true)
 |-- MUNICIPIO: string (nullable = false)
 |-- COORDENADA: string (nullable = false)



## Validação de dados

In [ ]:
#Criação do schema pandera

schema = pa.DataFrameSchema(
    columns = {
        'Data Medicao':pa.Column(pa.DateTime),
        'NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)':pa.Column(pa.Int),
        'PRECIPITACAO TOTAL - MENSAL (AUT)(mm)':pa.Column(pa.Float),
        'TEMPERATURA MEDIA - MENSAL (AUT)(°C)':pa.Column(pa.Float),
        'VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)':pa.Column(pa.Float),
        'MUNICIPIO':pa.Column(pa.String),
        'COORDENADA	':pa.Column(pa.String)
  }
)

##Tratamento de null

Tratamento PRECIPITACAO TOTAL - MENSAL (AUT)(mm)

In [ ]:
df_tratado.filter(isnull('PRECIPITACAO TOTAL - MENSAL (AUT)(mm)')).show()

+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+--------------------+
|Data Medicao|NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)|PRECIPITACAO TOTAL - MENSAL (AUT)(mm)|TEMPERATURA MEDIA - MENSAL (AUT)(°C)|VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)|VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)|           MUNICIPIO|          COORDENADA|
+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+--------------------+
|  2022-05-31|                                                       null|                                 null|                                null|        

In [ ]:
meses = [1,2,3,4,5,6,7,8,9,10,11,12]
precipitacao_normal = [115.9,	141.8,	107.2,	111.1,	117.1,	107.7,	112.7,	117.4,	128.7,	120.2,	99.4,	103.2]
medtemp_normal = [23.5,	23.2,	21.9,	19,	15.6,	13.1,	12.3,	13.8,	15.3,	17.9,	20.1,	22.2]
medvent_normal = [1.7,	None,	1.6,	1.6,	1.6,	1.8,	1.8,	1.9,	2,	1.9,	1.9,	1.8]

In [ ]:
col_substituindo = "PRECIPITACAO TOTAL - MENSAL (AUT)(mm)"
for i in range(len(meses)):
  mes_da_troca = meses[i]
  normal = precipitacao_normal[i]

  df_tratado = df_tratado.withColumn(col_substituindo, F.when\
                                    ((F.month("Data Medicao") == mes_da_troca) & (F.col(col_substituindo).isNull()), normal).otherwise(F.col(col_substituindo)))

Tratamento TEMPERATURA MEDIA - MENSAL (AUT)(°C)

In [ ]:
df_tratado.filter(isnull('TEMPERATURA MEDIA - MENSAL (AUT)(°C)')).show()

+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+--------------------+
|Data Medicao|NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)|PRECIPITACAO TOTAL - MENSAL (AUT)(mm)|TEMPERATURA MEDIA - MENSAL (AUT)(°C)|VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)|VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)|           MUNICIPIO|          COORDENADA|
+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+--------------------+
|  2022-05-31|                                                       null|                                117.1|                                null|        

In [ ]:
col_substituindo = "TEMPERATURA MEDIA - MENSAL (AUT)(°C)"
for i in range(len(meses)):
  mes_da_troca = meses[i]
  normal = medtemp_normal[i]

  df_tratado = df_tratado.withColumn(col_substituindo, F.when((F.month("Data Medicao") == mes_da_troca) & (F.col(col_substituindo).isNull()), normal).otherwise(F.col(col_substituindo)))


Tratamento VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)

In [ ]:
df_tratado.filter(isnull('VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)')).show()

+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+--------------------+
|Data Medicao|NUMERO DE DIAS COM PRECIPITACAO PLUV - MENSAL (AUT)(numero)|PRECIPITACAO TOTAL - MENSAL (AUT)(mm)|TEMPERATURA MEDIA - MENSAL (AUT)(°C)|VENTO - VELOCIDADE MAXIMA MENSAL (AUT)(Km/h)|VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)|           MUNICIPIO|          COORDENADA|
+------------+-----------------------------------------------------------+-------------------------------------+------------------------------------+--------------------------------------------+-------------------------------------------+--------------------+--------------------+
|  2022-05-31|                                                       null|                                117.1|                                15.6|        

In [ ]:
col_substituindo = "VENTO - VELOCIDADE MEDIA MENSAL (AUT)(Km/h)"
for i in range(len(meses)):
  if medvent_normal == None:
    pass
    mes_da_troca = meses[i]
    normal = (medvent_normal[i] * 3.6)

    df_tratado = df_tratado.withColumn(col_substituindo, F.when((F.month("Data Medicao") == mes_da_troca) & (F.col(col_substituindo).isNull()), normal).otherwise(F.col(col_substituindo)))


##Load (Subindo df tratado para GCP)

In [ ]:
df_salvar = df_tratado.toPandas()

In [ ]:
gcs_path = 'gs://tcc_soulcode_g3/tratados_clima_mensal/sul/dados_A899_M_2010-01-01_2022-12-31_tratado.csv'
df_salvar.to_csv(gcs_path, sep = ';', index = False)